In [3]:
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "[INST] What is the meaning of {word} [/INST]",
)

llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs={
        "max_new_tokens": 250,
    },
)

chain = prompt | llm

chain.invoke({"word": "flutter"})

' Flutter is an open-source mobile application development framework created by Google. It is used to build natively compiled applications for mobile, web, and desktop from a single codebase. Flutter uses the Dart programming language and provides a fast development, testing, and deployment experience with a rich set of widgets and libraries. It is known for its reactive UI, expressive and flexible UI toolkit, and the ability to deliver high-performance apps. Flutter compiles the code to native ARM machine code, which results in fast and smooth app performance.'